# Ligand Parameterization

Molecular mechanics forcefields need to be told how to treat each atom via a set of parameters. If there is a molecule (residue) in your system that your forcefield of choice does not already have parameters, you will need to build these yourself. In this example, we will see how QMzyme can automate the calculations required to parameterize a ligand in line with the RESP procedure (J. Phys. Chem. 1993, 97, 40, 10269–1028). 

In [1]:
from QMzyme import GenerateModel
from QMzyme.CalculateModel import QM_Method
from QMzyme.data import PDB

/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


### Initialize Model

In [2]:
# The initial pdb file should be preparred prior (hydrogens must be present).

pdb_file = PDB # here we are using package data. 
model = GenerateModel(PDB)

### Designate Ligand Region 

In [3]:
model.set_region(name='EQU', selection='resid 263 and resname EQU')
print(model.EQU)

<QMzymeRegion EQU contains 37 atom(s) and 1 residue(s)>


### Build the QM Method

In [4]:
# For the purpose of this example we will forgo geometry optimization and only perform a single point energy 
# calculation and population analysis at the level used in the original RESP procedure (J. Phys. Chem. 1993, 97, 40, 10269–1028).

qm_method = QM_Method(
    basis_set='6-31G*', 
    functional='HF', 
    qm_input='SCF=Tight Pop=MK IOp(6/33=2,6/42=6,6/43=20)', 
    program='gaussian'
)

### Assign QM Method to Region

In [5]:
# Now we assign this method to our QMzymeRegion EQU
# We also need to specify the charge and multiplicity (mult) because QMzyme currently only guesses charges of standard amino acids.

qm_method.assign_to_region(region=model.EQU, charge=-1, mult=1)

### Write Calculation Input File

In [6]:
# Back to our Model

# QMzyme will know we only have one region with a calculation method set, 
# so it will logically create the input file for that scenario

model.write_input()

File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/EQU.com created.
